In [1]:
import numpy as np
import sys 
import os
import matplotlib.pyplot as plt
import copy 
import tqdm
#import pandas as pd

In [62]:
# box_name = 'box02'
# subbox_name = 'subbox02'
# peak_name = '0'

# path = '/storage/filament/result/cluster_3d/40Mpc/density_temp/' + box_name + '/' +  subbox_name + '/label/'

path = '/home/seungwoo-ha/Works/data/'
peak_name = '0'
filament_conneted = np.fromfile(path + str(peak_name) ,dtype='int32')
filament_conneted = filament_conneted.reshape([101,101,101])
box_size = 8 
center_point = 50

coords_x = []
coords_y = []
coords_z = []

# x = 42
plane = filament_conneted[center_point - box_size, center_point-box_size : center_point+box_size+1  ,center_point-box_size : center_point+box_size+1 ]
coords = np.array(np.where(plane==1))

for n in range(coords.shape[1]):
    coords_1 = center_point + coords[0,n] - box_size
    coords_2 = center_point + coords[1,n] - box_size 

    coords_x.append(center_point-box_size)
    coords_y.append(coords_1)
    coords_z.append(coords_2)

# x = 58 
plane = filament_conneted[center_point + box_size, center_point-box_size : center_point+box_size+1  ,center_point-box_size : center_point+box_size+1 ]
coords = np.array(np.where(plane==1))

for n in range(coords.shape[1]):
    coords_1 = center_point + coords[0,n] - box_size
    coords_2 = center_point + coords[1,n] - box_size 

    coords_x.append(center_point+box_size)
    coords_y.append(coords_1)
    coords_z.append(coords_2)

# y = 42
plane = filament_conneted[center_point-box_size : center_point+box_size+1, center_point - box_size  ,center_point-box_size : center_point+box_size+1 ]
coords = np.array(np.where(plane==1))

for n in range(coords.shape[1]):
    coords_1 = center_point + coords[0,n] - box_size
    coords_2 = center_point + coords[1,n] - box_size 

    coords_x.append(coords_1)
    coords_y.append(center_point-box_size)
    coords_z.append(coords_2)    

# y = 58
plane = filament_conneted[center_point-box_size : center_point+box_size+1, center_point + box_size  ,center_point-box_size : center_point+box_size+1 ]
coords = np.array(np.where(plane==1))

for n in range(coords.shape[1]):
    coords_1 = center_point + coords[0,n] - box_size
    coords_2 = center_point + coords[1,n] - box_size 

    coords_x.append(coords_1)
    coords_y.append(center_point+box_size)
    coords_z.append(coords_2)       

# z = 42
plane = filament_conneted[center_point-box_size : center_point+box_size+1, center_point-box_size : center_point+box_size+1 , center_point - box_size  ]
coords = np.array(np.where(plane==1))

for n in range(coords.shape[1]):
    coords_1 = center_point + coords[0,n] - box_size
    coords_2 = center_point + coords[1,n] - box_size 

    coords_x.append(coords_1)
    coords_y.append(coords_2)
    coords_z.append(center_point-box_size)    

# z = 58
plane = filament_conneted[center_point-box_size : center_point+box_size+1 ,center_point-box_size : center_point+box_size+1 ,center_point + box_size]
coords = np.array(np.where(plane==1))

for n in range(coords.shape[1]):
    coords_1 = center_point + coords[0,n] - box_size
    coords_2 = center_point + coords[1,n] - box_size 

    coords_x.append(coords_1)
    coords_y.append(coords_2)
    coords_z.append(center_point+box_size)   
    
coords = np.array(list(zip(coords_x,coords_y,coords_z)))

filament_conneted[center_point - box_size+1 : center_point + box_size,center_point - box_size+1 : center_point + box_size,center_point - box_size+1 : center_point + box_size] = 0

            
            
filament = ()    
num = 0

filament_x = {}
filament_y = {}
filament_z = {}
# new filament!
#for n in range(coords.shape[0]):
for n in range(5):

    ref_x = coords[n,0]
    ref_y = coords[n,1]
    ref_z = coords[n,2]

    previous_x = ref_x
    previous_y = ref_y
    previous_z = ref_z

    filament_x[str(num)] = [previous_x]
    filament_y[str(num)] = [previous_y]
    filament_z[str(num)] = [previous_z]

    while True:

        candidate_coords_filament = find_candidate_coords(previous_x,previous_y,previous_z,filament_x,filament_y,filament_z,num)
        print("a",num,previous_x,previous_y,previous_z)
        if candidate_coords_filament.shape[0] == 0:

            filament_zip = list(zip(filament_x[str(num)],filament_y[str(num)],filament_z[str(num)]))
            filament = filament + (num,(filament_zip))            
            num = num + 1

            break

        elif candidate_coords_filament.shape[0] == 1:

            previous_x = candidate_coords_filament[0,0]
            previous_y = candidate_coords_filament[0,1]
            previous_z = candidate_coords_filament[0,2]

            filament_x[str(num)].append(previous_x)
            filament_y[str(num)].append(previous_y)
            filament_z[str(num)].append(previous_z)


        elif candidate_coords_filament.shape[0] >= 2:


            for nn in range(candidate_coords_filament.shape[0]):
     
                filament_x[str(num+nn)] = filament_x[str(num)]
                filament_y[str(num+nn)] = filament_y[str(num)]
                filament_z[str(num+nn)] = filament_z[str(num)]
                
            for nn in range(candidate_coords_filament.shape[0]):
     
                num = num + nn

                previous_x = candidate_coords_filament[nn,0]
                previous_y = candidate_coords_filament[nn,1]
                previous_z = candidate_coords_filament[nn,2]

                candidate_coords_filament = find_candidate_coords(previous_x,previous_y,previous_z,filament_x,filament_y,filament_z,num)

                while True:       
                    print("b",num,previous_x,previous_y,previous_z)                    
                    if candidate_coords_filament.shape[0] == 1:
                        previous_x = candidate_coords_filament[0,0]
                        previous_y = candidate_coords_filament[0,1]
                        previous_z = candidate_coords_filament[0,2]

                        filament_x[str(num)].append(previous_x)
                        filament_y[str(num)].append(previous_y)
                        filament_z[str(num)].append(previous_z)


                        candidate_coords_filament = find_candidate_coords(previous_x,previous_y,previous_z,filament_x,filament_y,filament_z,num)


                    else:

                        filament_zip = list(zip(filament_x[str(num)],filament_y[str(num)],filament_z[str(num)]))
                        filament = filament + (num,(filament_zip))   
                        num = num+1   
                        break
                    continue
                break
            break
        


a 0 42 58 43
b 0 41 57 44
b 0 41 57 44
a 1 58 51 43
a 1 59 51 42
b 1 59 51 42
a 2 48 42 46
a 2 47 41 45
b 2 46 40 44
a 3 54 42 47
a 3 54 41 46
b 3 54 41 46
a 4 42 58 43
b 4 41 57 44
b 4 41 57 44


In [63]:
filament_x

{'0': [42, 41],
 '1': [58, 59],
 '2': [48, 47],
 '3': [54, 54],
 '4': [42, 41],
 '5': [42, 41],
 '6': [54, 54]}

In [2]:
def find_candidate_coords(previous_x,previous_y,previous_z,filament_x,filament_y,filament_z,num):
    test_box = filament_conneted[previous_x-1:previous_x+2,previous_y-1:previous_y+2,previous_z-1:previous_z+2]
    candidate_set = np.array(np.where(test_box==1)).T

    ## candidate_set coords convert into real coords       
    for j in range(candidate_set.shape[0]):
        candidate_set[j][0] = candidate_set[j][0] - 1 + previous_x
        candidate_set[j][1] = candidate_set[j][1] - 1 + previous_y
        candidate_set[j][2] = candidate_set[j][2] - 1 + previous_z

    x_candidate = []
    y_candidate = []
    z_candidate = []

    filament_array = np.array(list(zip(filament_x[str(num)],filament_y[str(num)],filament_z[str(num)])))
    for n in range(filament_array.shape[0]):
        tmp_x = filament_x[str(num)][n]
        tmp_y = filament_y[str(num)][n]
        tmp_z = filament_z[str(num)][n]
        for m in range(candidate_set.shape[0]):
            if candidate_set[m][0] == tmp_x and candidate_set[m][1] == tmp_y and candidate_set[m][2] == tmp_z:
                pass
            else:
                x_candidate.append(candidate_set[m][0])
                y_candidate.append(candidate_set[m][1])
                z_candidate.append(candidate_set[m][2])

    candidate_coords_filament = np.array(list(zip(x_candidate,y_candidate,z_candidate)))
    return candidate_coords_filament

In [16]:
box_list = ['box01','box02']
subbox_list = ['subbox01','subbox02','subbox03','subbox04','subbox05','subbox06','subbox07','subbox08']

for box_name in box_list:
    for subbox_name in subbox_list:
        path = '/storage/filament/result/cluster_3d/40Mpc/density_temp/' + box_name + '/' +  subbox_name + '/label/'
        for peak_name in np.sort(os.listdir(path)):
            test = np.fromfile(path + str(peak_name) ,dtype='int32')
            test_re = test.reshape([101,101,101])
            box_size = 8 
            center_point = 50

            coords_x = []
            coords_y = []
            coords_z = []

            # x = 42
            plane = test_re[center_point - box_size, center_point-box_size : center_point+box_size  ,center_point-box_size : center_point+box_size ]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(center_point-box_size)
                coords_y.append(coords_1)
                coords_z.append(coords_2)

            # x = 58 
            plane = test_re[center_point + box_size, center_point-box_size : center_point+box_size  ,center_point-box_size : center_point+box_size ]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(center_point+box_size)
                coords_y.append(coords_1)
                coords_z.append(coords_2)

            # y = 42
            plane = test_re[center_point-box_size : center_point+box_size,center_point - box_size  ,center_point-box_size : center_point+box_size ]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(coords_1)
                coords_y.append(center_point-box_size)
                coords_z.append(coords_2)    

            # y = 58
            plane = test_re[center_point-box_size : center_point+box_size,center_point + box_size  ,center_point-box_size : center_point+box_size ]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(coords_1)
                coords_y.append(center_point+box_size)
                coords_z.append(coords_2)       

            # z = 42
            plane = test_re[center_point-box_size : center_point+box_size,center_point-box_size : center_point+box_size,center_point - box_size  ]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(coords_1)
                coords_y.append(coords_2)
                coords_z.append(center_point-box_size)    

            # z = 58
            plane = test_re[center_point-box_size : center_point+box_size,center_point-box_size : center_point+box_size,center_point + box_size]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(coords_1)
                coords_y.append(coords_2)
                coords_z.append(center_point+box_size)   
                
            coords = np.array(list(zip(coords_x,coords_y,coords_z)))
            
            
            
            ###
            
            filament = ()    

            for n in range(coords.shape[0]):
                filament_x = []
                filament_y = []
                filament_z = []

                ref_x = coords[n,0]
                ref_y = coords[n,1]
                ref_z = coords[n,2]

                previous_x = ref_x
                previous_y = ref_y
                previous_z = ref_z

                filament_x.append(previous_x)
                filament_y.append(previous_y)
                filament_z.append(previous_z)
                ### 필라멘트 1개의 integration
                for i in range(50):

                    test_box = test_re[previous_x-1:previous_x+2,previous_y-1:previous_y+2,previous_z-1:previous_z+2]

                    candidate_set = np.array(np.where(test_box==1)).T
                    
                    candidate_set = pcandidate_set - 1
                    if candidate_set.shape[0] <= 2:
                        break
                        candidate_set 
                    # #print(candidate_set.shape)
                    # if candidate_set.shape[0] == 0:
                    #     break
                    # elif candidate_set.shape[0] == 3:
                    #     candidate_x_list = []
                    #     candidate_y_list = []
                    #     candidate_z_list = []

                    #     distance_list = []
                    #     ### 박스
                    #     for j in range(candidate_set.shape[0]):

                    #         candidate_x = previous_x + (candidate_set[j][0]-1)
                    #         candidate_y = previous_y + (candidate_set[j][1]-1)
                    #         candidate_z = previous_z + (candidate_set[j][2]-1)

                    #         candidate_x_list.append(candidate_x)
                    #         candidate_y_list.append(candidate_y)
                    #         candidate_z_list.append(candidate_z)

                    #         distance_list.append(np.sqrt( (center_point - candidate_x)**2  + (center_point - candidate_y)**2 + (center_point - candidate_z)**2))

                    #     forward_point = np.argmax(distance_list)   

                    #     tmp_x = candidate_x_list[forward_point]
                    #     tmp_y = candidate_y_list[forward_point]
                    #     tmp_z = candidate_z_list[forward_point]

                    #     if tmp_x == previous_x and tmp_y == previous_y and tmp_z == previous_z:
                    #         break
                    #     else:
                    #         previous_x = tmp_x
                    #         previous_y = tmp_y
                    #         previous_z = tmp_z

                    #         filament_x.append(previous_x)
                    #         filament_y.append(previous_y)
                    #         filament_z.append(previous_z)
                    #     #else:
                    # elif candidate_set.shape[3] > 3:


                        

                filament_zip = list(zip(filament_x,filament_y,filament_z))
                filament = filament + (n,(filament_zip))
                
                save_path = '/storage/filament/result/cluster_3d/40Mpc/density_temp/' + box_name + '/' + subbox_name + '/filament/' + peak_name + '/'                             


                for i in range(len(filament)):
                    tmp = filament[i]

                    if type(tmp) == int:
                        if not os.path.isdir(save_path  + '/'):
                            os.makedirs(save_path + '/')
                        os.chdir(save_path + '/')    
                        tmp_filament = filament[i+1]
                        np.savetxt(str(tmp),tmp_filament,fmt='%3i')
                    else:
                        continue

        
    

In [15]:
save_path

'/storage/filament/result/cluster_3d/40Mpc/density_temp/box02/subbox06/filament/0/'

In [132]:
filament

(0,
 [(42, 50, 47),
  (41, 50, 46),
  (40, 50, 46),
  (39, 51, 46),
  (38, 50, 46),
  (37, 50, 45),
  (36, 49, 45),
  (35, 49, 44),
  (34, 49, 44),
  (33, 48, 44),
  (32, 48, 43),
  (31, 48, 43),
  (30, 47, 43),
  (29, 47, 42),
  (28, 47, 42),
  (27, 46, 41),
  (26, 46, 41),
  (25, 46, 40),
  (24, 46, 40),
  (23, 46, 40),
  (22, 45, 39),
  (21, 45, 39),
  (20, 45, 38),
  (19, 44, 37),
  (18, 45, 37),
  (17, 45, 36),
  (16, 45, 35),
  (15, 45, 35)],
 1,
 [(58, 42, 42), (57, 42, 41)],
 2,
 [(58, 43, 46), (59, 42, 45), (59, 42, 44), (59, 42, 43)],
 3,
 [(52, 42, 56), (53, 41, 57)],
 4,
 [(52, 42, 57), (53, 41, 57)],
 5,
 [(53, 42, 51),
  (53, 41, 52),
  (53, 40, 53),
  (54, 39, 54),
  (55, 38, 53),
  (56, 37, 52),
  (57, 36, 51),
  (57, 35, 50),
  (57, 34, 49),
  (58, 34, 48),
  (59, 34, 47),
  (60, 33, 47),
  (61, 33, 46),
  (62, 33, 45),
  (63, 32, 46),
  (64, 31, 46),
  (65, 30, 46),
  (66, 30, 45),
  (67, 29, 44),
  (68, 29, 43),
  (68, 29, 42),
  (69, 28, 41),
  (70, 28, 40),
  (71, 

In [17]:
box_list = ['box01_add','box02_add']

for box_name in box_list:
    box_path = '/storage/filament/result/cluster_3d/40Mpc/density_temp/' + box_name + '/'
    for subbox_name in os.listdir(box_path):
        path = '/storage/filament/result/cluster_3d/40Mpc/density_temp/' + box_name + '/' +  subbox_name + '/label/'
        for peak_name in np.sort(os.listdir(path)):
            test = np.fromfile(path + '0' ,dtype='int32')
            test_re = test.reshape([101,101,101])
            box_size = 8 
            center_point = 50

            coords_x = []
            coords_y = []
            coords_z = []

            # x = 42
            plane = test_re[center_point - box_size, center_point-box_size : center_point+box_size  ,center_point-box_size : center_point+box_size ]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(center_point-box_size)
                coords_y.append(coords_1)
                coords_z.append(coords_2)

            # x = 58 
            plane = test_re[center_point + box_size, center_point-box_size : center_point+box_size  ,center_point-box_size : center_point+box_size ]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(center_point+box_size)
                coords_y.append(coords_1)
                coords_z.append(coords_2)

            # y = 42
            plane = test_re[center_point-box_size : center_point+box_size,center_point - box_size  ,center_point-box_size : center_point+box_size ]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(coords_1)
                coords_y.append(center_point-box_size)
                coords_z.append(coords_2)    

            # y = 58
            plane = test_re[center_point-box_size : center_point+box_size,center_point + box_size  ,center_point-box_size : center_point+box_size ]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(coords_1)
                coords_y.append(center_point+box_size)
                coords_z.append(coords_2)       

            # z = 42
            plane = test_re[center_point-box_size : center_point+box_size,center_point-box_size : center_point+box_size,center_point - box_size  ]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(coords_1)
                coords_y.append(coords_2)
                coords_z.append(center_point-box_size)    

            # z = 58
            plane = test_re[center_point-box_size : center_point+box_size,center_point-box_size : center_point+box_size,center_point + box_size]
            coords = np.array(np.where(plane==1))

            for n in range(coords.shape[1]):
                coords_1 = center_point + coords[0,n] - box_size
                coords_2 = center_point + coords[1,n] - box_size 

                coords_x.append(coords_1)
                coords_y.append(coords_2)
                coords_z.append(center_point+box_size)   
                
            coords = np.array(list(zip(coords_x,coords_y,coords_z)))
            
            
            
            ###
            
            filament = ()    

            for n in range(coords.shape[0]):
                filament_x = []
                filament_y = []
                filament_z = []

                ref_x = coords[n,0]
                ref_y = coords[n,1]
                ref_z = coords[n,2]

                previous_x = ref_x
                previous_y = ref_y
                previous_z = ref_z

                filament_x.append(previous_x)
                filament_y.append(previous_y)
                filament_z.append(previous_z)
                ### 필라멘트 1개의 integration
                for i in range(50):

                    test_box = test_re[previous_x-1:previous_x+2,previous_y-1:previous_y+2,previous_z-1:previous_z+2]

                    candidate_set = np.array(np.where(test_box==1)).T
                    #print(candidate_set.shape)
                    if candidate_set.shape[0] ==0:
                        break
                    else:
                        candidate_x_list = []
                        candidate_y_list = []
                        candidate_z_list = []

                        distance_list = []
                        ### 박스
                        for j in range(candidate_set.shape[0]):

                            candidate_x = previous_x + (candidate_set[j][0]-1)
                            candidate_y = previous_y + (candidate_set[j][1]-1)
                            candidate_z = previous_z + (candidate_set[j][2]-1)

                            candidate_x_list.append(candidate_x)
                            candidate_y_list.append(candidate_y)
                            candidate_z_list.append(candidate_z)

                            distance_list.append(np.sqrt( (center_point - candidate_x)**2  + (center_point - candidate_y)**2 + (center_point - candidate_z)**2))

                        forward_point = np.argmax(distance_list)   

                        tmp_x = candidate_x_list[forward_point]
                        tmp_y = candidate_y_list[forward_point]
                        tmp_z = candidate_z_list[forward_point]

                        if tmp_x == previous_x and tmp_y == previous_y and tmp_z == previous_z:
                            break
                        else:
                            previous_x = tmp_x
                            previous_y = tmp_y
                            previous_z = tmp_z

                            filament_x.append(previous_x)
                            filament_y.append(previous_y)
                            filament_z.append(previous_z)
                        #else:


                filament_zip = list(zip(filament_x,filament_y,filament_z))
                filament = filament + (n,(filament_zip))
                
                save_path = '/storage/filament/result/cluster_3d/40Mpc/density_temp/' + box_name + '/' + subbox_name + '/filament/'                              


                for i in range(len(filament)):
                    tmp = filament[i]

                    if type(tmp) == int:
                        if not os.path.isdir(save_path  + '/'):
                            os.makedirs(save_path + '/')
                        os.chdir(save_path + '/')    
                        tmp_filament = filament[i+1]
                        np.savetxt(str(tmp),tmp_filament,fmt='%3i')
                    else:
                        continue

        
        